# WORD EMBEDDING

## 参考资料

[什么是 word embedding?](https://www.zhihu.com/question/32275069/answer/61059440)

[WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py)

[YJango的Word Embedding--介绍](https://zhuanlan.zhihu.com/p/27830489)

## 概念解析

## 代码解析

与我们制作独热向量（one-hot vectors）时候为每个单词定义一个唯一索引类似，在使用词嵌入的时候我们也需要为每个单词定义索引，方便查找。

词嵌入被储存为了一个 $ |V|×D $ 的矩阵，其中 $ D $ 是嵌入的维度，所以某单词的索引为 $ i $，那么它就被嵌入在矩阵的第 $ i $ 行中。在代码中，从单词到索引的映射是一个名叫 word_to_ix 的字典。

在 PyTorch 中，torch.nn.Embedding 可以完成词嵌入功能，包含两个参数：词表大小和嵌入维度。

使用 torch.LongTensor 进行索引（这是一个 64-bit integer，带符号数）。

In [1]:
# Author: Robert Guthrie
# 补充：Dr_David_S

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)  # 设定随机种子，返回一个torch.Generator对象

现在我们设置一个字典，用最简单的 hello world 做例子：

In [2]:
word_to_ix = {"hello": 0, "world": 1}
word_to_ix

{'hello': 0, 'world': 1}

使用 torch.nn.Embedding 方法准备进行词嵌入，参数的意义是：有2个词语，5个维度。

In [3]:
embeds = nn.Embedding(num_embeddings=2, embedding_dim=5)  # 2 words in vocab, 5 dimensional embeddings
embeds

Embedding(2, 5)

现在将 hello 对应的值 0 转换为一个 longtensor 类型,两种方法都行。

In [4]:
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
lookup_tensor

tensor([0])

In [5]:
lookup_tensor_test = torch.LongTensor([word_to_ix["hello"]])
lookup_tensor_test

tensor([0])

现在使用之前定义的 embeds 模型对 hello 这个次进行词嵌入（或者说转换为词向量）：

In [6]:
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


原来的代码到此就结束了，但是这里我们希望再试试对 world 的词嵌入：

In [7]:
lookup_tensor2 = torch.tensor([word_to_ix["world"]], dtype=torch.long)
lookup_tensor2

tensor([1])

In [8]:
world_embed = embeds(lookup_tensor2)
print(world_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


从上述结果来看，似乎对于 hello 的 embedding 没有涉及到 world？

事实上，embeds只是一个随机生成的矩阵，目前其中的向量并不能代表任何意义，如下：

In [9]:
embeds.weight

Parameter containing:
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]], requires_grad=True)

>nn.Embedding.weight初始化分布符合标准正态分布 $ N(0,1) $，即均值 $\mu=0$，方差 $\sigma=1$ 的正态分布。
>
>因为这里的初始化参数只设定了两个单词，所以其中：
>
>某一行代表单词 'hello'
>
>另一行代表单词 'world'

## 一个 N-Gram 语言模型

之前我们的 embeds 矩阵的初始值是随机生成的，现在我们将要计算一些训练样例的损失函数，然后使用反向传播去更新参数。

### 文本处理

首先设定参数：

In [10]:
CONTEXT_SIZE = 2  # 指上下文的size
EMBEDDING_DIM = 10

其中，上下文是指目标单词的前 $ n $ 个单词，或者后 $ n $ 个单词，或者前后都有总共 $ n $ 个单词。

在本例中，我们使用的是莎士比亚的十四行诗，使用空格分词：

In [11]:
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

>SONNET 2：
>
>当四十个冬天围攻你的朱颜，
>
>在你美的园地挖下深的战壕，
>
>你青春的华服，那么被人艳羡，
>
>将成褴褛的败絮，谁也不要瞧：
>
>那时人若问起你的美在何处，
>
>哪里是你那少壮年华的宝藏，
>
>你说，"在我这双深陷的眼眶里，
>
>是贪婪的羞耻，和无益的颂扬。"
>
>你的美的用途会更值得赞美，
>
>如果你能够说，"我这宁馨小童
>
>将总结我的账，宽恕我的老迈，"
>
>证实他的美在继承你的血统！
>
>这将使你在衰老的暮年更生，
>
>并使你垂冷的血液感到重温。
>

In [12]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

按道理，我们在训练之前输入的数据应当是有标记的，我们暂时忽略这个事情（后面会说明）。

然后我们要建立一个元组（tuple），元组的格式为：

$$ ([word_{i-2}, word_{i-1}], word_{i}) $$

其中 $word_{i}$ 就是目标词 $ target word $ 

根据以上方法遍历整首诗，然后生成多个元组，放入列表 trigrams 中。

In [13]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


set() 函数创建一个无序不重复元素集，可以剔除掉重复的元素，同时可能可以在部分模型如 RNN 中打乱文档上下文，忽略掉 RNN 权重偏向后输入的词语的缺点。

In [14]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

### 搭建网络

接下来我们构建一个网络，名叫 NGramLanguageModeler：

In [15]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

从上述网络来看，init 中存在三个层，分别是：

- 词嵌入层（embedding层）
- 第一个全连接层（linear1）
- 第二个全连接层（linear2）

算上前向传播的过程就是：

- 词嵌入层
- 转换为一维
- 第一个全连接层
- ReLU激活
- 第二个全连接层
- log_softmax输出

### 设置损失函数和优化函数

- 使用NLLLoss()，这个函数其实就是一个不带 log_softmax 的交叉熵损失函数。

- 使用optim.SGD()，最传统的随机梯度下降，没什么好说的。

需要注意的是，model 的输入参数有三个，第一个就是去重后的上下文三词元组数量，第二三个则是我们设定好的 CONTEXT_SIZE 和 EMBEDDING_DIM。

In [16]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

### 训练模型

注意：
```Python
for context, target in trigrams
```
这句话指的是我们的训练样本是上下文（其实只有上文），标签是当前词。这也是为什么之前没有标记的原因。

In [44]:
trigrams

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege'),
 (['shall', 'besiege'], 'thy'),
 (['besiege', 'thy'], 'brow,'),
 (['thy', 'brow,'], 'And'),
 (['brow,', 'And'], 'dig'),
 (['And', 'dig'], 'deep'),
 (['dig', 'deep'], 'trenches'),
 (['deep', 'trenches'], 'in'),
 (['trenches', 'in'], 'thy'),
 (['in', 'thy'], "beauty's"),
 (['thy', "beauty's"], 'field,'),
 (["beauty's", 'field,'], 'Thy'),
 (['field,', 'Thy'], "youth's"),
 (['Thy', "youth's"], 'proud'),
 (["youth's", 'proud'], 'livery'),
 (['proud', 'livery'], 'so'),
 (['livery', 'so'], 'gazed'),
 (['so', 'gazed'], 'on'),
 (['gazed', 'on'], 'now,'),
 (['on', 'now,'], 'Will'),
 (['now,', 'Will'], 'be'),
 (['Will', 'be'], 'a'),
 (['be', 'a'], "totter'd"),
 (['a', "totter'd"], 'weed'),
 (["totter'd", 'weed'], 'of'),
 (['weed', 'of'], 'small'),
 (['of', 'small'], 'worth'),
 (['small', 'worth'], 'held:'),
 (['worth', 'held:'], 'Then'),
 (['held:', 'Then'], 'being'),
 (['Then', 'being'], 'asked

In [17]:
for epoch in range(100):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        # 这一步把上下文单词转换为字典中对应单词的值，作为tensor
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[519.6867213249207, 517.070152759552, 514.4715573787689, 511.8875916004181, 509.3183901309967, 506.76496982574463, 504.226984500885, 501.7028648853302, 499.19131565093994, 496.6922929286957, 494.2053587436676, 491.7301506996155, 489.26255536079407, 486.80488872528076, 484.3552463054657, 481.91312432289124, 479.4765045642853, 477.04330587387085, 474.6125531196594, 472.185026884079, 469.7598648071289, 467.3372013568878, 464.91707134246826, 462.4967677593231, 460.0782268047333, 457.659059047699, 455.239577293396, 452.81758403778076, 450.3921182155609, 447.9660198688507, 445.538423538208, 443.1092698574066, 440.6768057346344, 438.2413046360016, 435.8006672859192, 433.3557085990906, 430.90441489219666, 428.4492378234863, 425.9890410900116, 423.52271246910095, 421.04992413520813, 418.56983613967896, 416.0808434486389, 413.58485221862793, 411.0811542272568, 408.5684061050415, 406.0478866100311, 403.5204122066498, 400.98438131809235, 398.44108045101166, 395.8876748085022, 393.32611668109894, 3

现在我们得到了一个损失逐渐降低的模型，可以看出这个模型的结果是收敛的，如果继续训练下去，可能会更准确，但是到此为止吧。

## 练习：计算词嵌入：CBOW模型

连续词袋模型（Continuous Bag-of-Words model ），又被称为CBOW模型，经常用于自然语言处理深度学习。

CBOW模型使用一段文本的中间词作为目标词，即给出目标词的上下文单词作为训练数据。CBOW去除了上下文各词语的词序信息，使用的是上下文各词的词向量的平均值。

CBOW对于一段长度为 $ n $ 的训练样本：$ w_{i-(n-1)}, ...,w_i $ ，其输入为: $$ x = \frac {1}{n - 1}\sum_{w_j \in c}e(w_j) $$

其中 $ w $ 是目标词，即 $ w_i $， $ c $ 是上下文，上式就是将输入求平均。

故CBOW模型根据上下文的表示，直接对目标词进行预测： $$ P(w|c) = \frac{\exp(e'(w)^Tx)}{\sum_{w'\in V}\exp \lgroup e'(w')^Tx\rgroup} $$ 

CBOW的目标是最大化 $$ \sum_{(w,c)\in D}\log P(w|c) $$

其中 $D$ 指的是整段语料（或者说整篇文章）。

要在PyTorch中实现这个模型，需要注意的是：

- 考虑一下需要定义哪些参数
- 确保你清楚每一步需要哪些形状的矩阵，使用 .view() 函数来 reshape 矩阵

### 代码示例

代码示例如下，第一步，先按空格分隔语料中的单词，其中 2 表示上文两个单词加下文两个单词。

In [19]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

使用 set() 函数将分词唯一化并打乱顺序，然后建立一个名叫 word_to_ix 的字典，字典的 key 是单词，value 是单词的索引。

接下来使用data列表来储存文本和目标了，把第 $i$ 个单词作为目标，把第$[i-2, i-1, i+1, i+2]$ 个单词作为上下文（特征），注意这里使用的是有序的语料。

In [24]:
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    # context保存目标单词上下文，分别2个单词
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    # target保存目标单词
    target = raw_text[i]
    # 存入data列表中
    data.append((context, target))
    
print(data[:5])
print(len(data))

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
58


建立 CBOW 类。

In [37]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    """从字典中抽取word对应的value，从
    
    输入：
    context:上下文列表（四个单词）
    word_to_ix:单词的字典
    
    输出：context中的单词对应的longtensor
    """
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


first_context = make_context_vector(data[0][0], word_to_ix)  # example
print(first_context)

tensor([20, 27, 26, 21])


这里给了一个例子，就是

```Python
make_context_vector(data[0][0], word_to_ix)  # example
```
简单分析一下：

```Python
data[0][0]
```
指的就是前面 data 列表中第一个tuple中的第一个元素，即:

In [38]:
print(data[0][0])

['We', 'are', 'to', 'study']


上述四个单词在字典 word_to_ix 中对应的值如下：

In [39]:
print(word_to_ix['We'])
print(word_to_ix['are'])
print(word_to_ix['to'])
print(word_to_ix['study'])

20
27
26
21


故其组成的 longtensor 自然就是 tensor([20, 27, 26, 21]) 啦~

现在，使用 torch.nn.Embedding 方法建立一个embedding模型，目标是58个词，每个词20维。

In [30]:
embeds = nn.Embedding(num_embeddings=58, embedding_dim=20)  # 58 words in vocab, 20 dimensional embeddings
embeds

Embedding(58, 20)

我们尝试使用上面的 embeds 模型将 first_context 向量化：

In [40]:
first_context_embed = embeds(first_context)
print(first_context_embed)

tensor([[-0.7294,  0.5238, -0.0033, -0.1415,  0.2730, -0.5259, -0.9415,  1.3637,
         -0.9278, -1.4225, -1.5137,  0.1082, -0.7336,  0.9111,  1.2902,  0.0399,
          0.2653,  0.6930,  1.0074,  0.0483],
        [-0.9999,  1.1559, -1.0390,  1.2571, -0.9196,  0.5568, -0.2764, -0.7657,
          1.5624,  0.9459,  0.1284,  0.7037, -0.9738, -1.6594,  0.9767,  0.1253,
         -0.2701, -0.5471,  0.9857,  1.0711],
        [ 0.6238, -0.0821,  0.1957,  0.6610,  0.1671, -1.4554, -0.7909,  0.3359,
          0.9374, -0.6158,  0.0269, -1.5926, -0.6403, -0.2427,  0.6413, -0.0834,
         -0.3291, -0.4984, -0.4400, -0.1743],
        [ 0.0564, -0.1179, -0.5898, -0.8935, -1.6001, -1.8835,  0.4199,  0.1604,
          1.1381, -0.5972,  0.1693,  0.3179,  0.8563,  1.2030,  0.0385,  1.2301,
         -1.4059,  0.3539, -1.1746, -1.3560]], grad_fn=<EmbeddingBackward>)


可以看出已经成功的完成了我们向量化的第一步实验，接下来的任务就是把所有的上下文和目标单词向量化，然后建立一个 CBOW 模型对这些向量进行训练。

顺便看看 embeds 的初始化，应当是随机初始化了一个符合标准正态分布 $ N(0,1) $的 58 行，20 列的矩阵。

In [47]:
embeds.weight

Parameter containing:
tensor([[-1.1007, -0.1941,  2.2328,  ..., -0.8124, -0.8811, -0.2669],
        [-0.2247,  0.3493, -0.4147,  ...,  0.9627, -0.2674, -1.2740],
        [-2.1191, -0.6800, -0.7205,  ..., -0.7623,  1.3541, -0.5149],
        ...,
        [-0.0924, -1.3570, -0.1421,  ...,  1.6972, -1.1635, -1.1725],
        [ 0.1829, -1.2565, -0.4906,  ..., -0.7481, -1.0837, -0.4727],
        [-0.5400,  0.8668, -1.8730,  ..., -1.2223,  0.6370, -1.0238]],
       requires_grad=True)